#### By: Peyman Shahidi
#### Created: Nov 7, 2025
#### Last Edit: Nov 7, 2025

<br>

In [1]:
#Python
import getpass
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import random 

## formatting number to appear comma separated and with two digits after decimal: e.g, 1000 shown as 1,000.00
pd.set_option('float_format', "{:,.2f}".format)

import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib.legend import Legend

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)

In [2]:
main_folder_path = ".."
input_data_path = f"{main_folder_path}/data"
output_data_path = f'{input_data_path}/computed_objects/fragmentationIndex'
output_plot_path = f"{main_folder_path}/writeup/plots"

In [3]:
# Create directories if they don't exist
import os

for path in [output_data_path, output_plot_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
def create_occupation_analysis(df, onet_occupation_code_var, onet_occupation_title_var):
    # Create occupation-level analysis for scatter plots
    # Group by occupation and calculate label fractions and task counts
    occupation_stats = []

    for (soc_code, occ_title), group in df.groupby([onet_occupation_code_var, onet_occupation_title_var]):
        num_tasks = group['Task ID'].nunique()
        # num_occupations = group[onet_occupation_code_var].nunique()
        total_tasks = len(group)
        
        manual_fraction = (group['label'] == 'Manual').sum() / total_tasks
        augmentation_fraction = (group['label'] == 'Augmentation').sum() / total_tasks  
        automation_fraction = (group['label'] == 'Automation').sum() / total_tasks
        ai_fraction = augmentation_fraction + automation_fraction
        gpt4_E0_fraction = (group['gpt4_exposure'] == 'E0').sum() / total_tasks
        gpt4_E1_fraction = (group['gpt4_exposure'] == 'E1').sum() / total_tasks
        gpt4_E2_fraction = (group['gpt4_exposure'] == 'E2').sum() / total_tasks
        gpt4_aiExposure_fraction = gpt4_E1_fraction + gpt4_E2_fraction
        human_E0_fraction = (group['human_labels'] == 'E0').sum() / total_tasks
        human_E1_fraction = (group['human_labels'] == 'E1').sum() / total_tasks
        human_E2_fraction = (group['human_labels'] == 'E2').sum() / total_tasks
        human_aiExposure_fraction = human_E1_fraction + human_E2_fraction

        
        occupation_stats.append({
            f'{onet_occupation_code_var}': soc_code,
            f'{onet_occupation_title_var}': occ_title,
            'num_tasks': num_tasks,
            # 'num_occupations': num_occupations,
            'manual_fraction': manual_fraction,
            'ai_fraction': ai_fraction,
            'augmentation_fraction': augmentation_fraction,
            'automation_fraction': automation_fraction,
            'gpt4_E0_fraction': gpt4_E0_fraction,
            'gpt4_E1_fraction': gpt4_E1_fraction,
            'gpt4_E2_fraction': gpt4_E2_fraction,
            'gpt4_aiExposure_fraction': gpt4_aiExposure_fraction,
            'human_E0_fraction': human_E0_fraction,
            'human_E1_fraction': human_E1_fraction,
            'human_E2_fraction': human_E2_fraction,
            'human_aiExposure_fraction': human_aiExposure_fraction
        })

    occupation_analysis = pd.DataFrame(occupation_stats)

    return occupation_analysis

In [5]:
# Read the merged data
merged_data = pd.read_csv(f"{input_data_path}/computed_objects/ONET_Eloundou_Anthropic_GPT/ONET_Eloundou_Anthropic_GPT.csv")

In [6]:
# # Drop the supplemental tasks
# merged_data = merged_data[merged_data['Task Type'] != 'Supplemental'].reset_index(drop=True)

# # Drop rows whose Occupation Title includes 'Teachers, Postsecondary'
# merged_data = merged_data[~merged_data['Occupation Title'].str.contains('Teachers, Postsecondary')].reset_index(drop=True)

In [7]:
# Define levels and variables
onet_levels = ['major', 'minor', 'broad', 'detailed']
onet_occupation_code_vars = ['Major_Group_Code', 'Minor_Group_Code', 'Broad_Occupation_Code', 'Detailed_Occupation_Code']
onet_occupation_title_vars = ['Major_Group_Title', 'Minor_Group_Title', 'Broad_Occupation_Title', 'Detailed_Occupation_Title']
dependent_var_list = ['ai_fraction', 'human_E1_fraction']#, 'human_aiExposure_fraction']#, 'gpt4_E1_fraction']

In [8]:
# Calculate fragmentation index
# For the sake of this exercise we treat all AI tasks as a single category

# Get occupation data
occupation_analysis = create_occupation_analysis(merged_data, 'O*NET-SOC Code', 'Occupation Title')

# Create is_ai column
fi_df = merged_data.copy()
fi_df['is_ai'] = fi_df['label'].isin(['Augmentation', 'Automation']).astype(int)


# Create next_is_ai column within occupation groups
fi_df['next_is_ai'] = fi_df.groupby(['O*NET-SOC Code', 'Occupation Title'])['is_ai'].shift(-1).fillna(0).astype(int)

# Calculate FI using incremental counter: only if current task and next task is AI do not increment FI
fi_df['fi_counter'] = 1
fi_df.loc[(fi_df['is_ai'] == 1) & (fi_df['next_is_ai'] == 1), 'fi_counter'] = 0

fi_df = fi_df.groupby(['O*NET-SOC Code', 'Occupation Title'])['fi_counter'].mean()
fi_df = fi_df.reset_index().rename(columns={'fi_counter': 'fragmentation_index'})

# Save fragmentation index data
fi_df.to_csv(f"{output_data_path}/fragmentationIndex.csv", index=False)
fi_df.head(10)

,O*NET-SOC Code,Occupation Title,fragmentation_index
0,11-1011.00,Chief Executives,0.97
1,11-1011.03,Chief Sustainability Officers,0.94
2,11-1021.00,General and Operations Managers,1.00
3,11-2011.00,Advertising and Promotions Managers,1.00
4,11-2021.00,Marketing Managers,1.00
5,11-2022.00,Sales Managers,1.00
6,11-3012.00,Administrative Services Managers,1.00
7,11-3021.00,Computer and Information Systems Managers,1.00
8,11-3031.00,Financial Managers,0.90
9,11-3031.01,Treasurers and Controllers,1.00


In [9]:
# Merge fragmentation index with occupation analysis
occupation_analysis = occupation_analysis.merge(fi_df, on=['O*NET-SOC Code', 'Occupation Title'], how='left')

# Save occupation analysis with fragmentation index
occupation_analysis.to_csv(f"{output_data_path}/occupation_analysis_with_fragmentationIndex.csv", index=False)

In [10]:
# First aggregate data at detailed_occupation level
my_onet_level = 'detailed'
onet_occupation_code_var = 'Detailed_Occupation_Code'
onet_occupation_title_var = 'Detailed_Occupation_Title'

# Read OG occupation analysis with SOC mappings
ONET = pd.read_csv(f"{input_data_path}/computed_objects/ONET_cleaned_tasks.csv")

# Keep only the relevant 
SOC_mappings = ONET[['O*NET-SOC Code', 'Occupation Title',
                     'Major_Group_Code', 'Major_Group_Title',
                     'Minor_Group_Code', 'Minor_Group_Title',
                     'Broad_Occupation_Code', 'Broad_Occupation_Title',
                     'Detailed_Occupation_Code', 'Detailed_Occupation_Title']].copy()
SOC_mappings = SOC_mappings.drop_duplicates(subset=['O*NET-SOC Code', onet_occupation_code_var])

# Merge SOC levels with the occupation analysis
occupation_analysis = occupation_analysis.merge(SOC_mappings, on=['O*NET-SOC Code', 'Occupation Title'], how='left')

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

ai_exposure_var = 'human_E1_fraction'

# Aggregate
occupation_analysis_aggregated = occupation_analysis.groupby(
    [onet_occupation_code_var, onet_occupation_title_var]
).agg({
    'fragmentation_index': 'mean',
    ai_exposure_var: 'mean',
    'ai_fraction': 'mean',
    'num_tasks': 'mean'
}).reset_index()

# Merge SOC levels for FE
occupation_analysis_aggregated = occupation_analysis_aggregated.merge(
    SOC_mappings, on=onet_occupation_code_var, how='left'
)

# Helper: compact table for selected vars
def coef_table(res, vars_):
    ci = res.conf_int().rename(columns={0: 'ci_low', 1: 'ci_high'})
    out = (pd.DataFrame({'coef': res.params, 'se': res.bse,
                         't': res.tvalues, 'p': res.pvalues})
           .join(ci)
           .loc[vars_])
    return out

vars_of_interest = ['fragmentation_index', ai_exposure_var, 'num_tasks']

# --- Model A: no FE ---
mod_a = smf.ols(
    formula=f'ai_fraction ~ fragmentation_index + {ai_exposure_var} + num_tasks',
    data=occupation_analysis_aggregated
).fit(cov_type='HC3')
# print("\n=== Model A (no FE) — FULL SUMMARY ===")
# print(mod_a.summary2())   # full details
print("\n\n=== Model A — selected coefficients ===")
print(coef_table(mod_a, vars_of_interest))

# --- Model B (Major group FE) ---
mod_b_major = smf.ols(
    formula=f'ai_fraction ~ fragmentation_index + {ai_exposure_var} + C(Major_Group_Code) + num_tasks',
    data=occupation_analysis_aggregated
).fit(cov_type='HC3')
# print("\n=== Model B (Major FE) — FULL SUMMARY ===")
# print(mod_b_major.summary2())
print("\n\n=== Model B (Major FE) — selected coefficients ===")
print(coef_table(mod_b_major, vars_of_interest))

# --- Model C (Minor group FE) ---
mod_c_minor = smf.ols(
    formula=f'ai_fraction ~ fragmentation_index + {ai_exposure_var} + C(Minor_Group_Code) + num_tasks',
    data=occupation_analysis_aggregated
).fit(cov_type='HC3')
# print("\n=== Model C (Minor FE) — FULL SUMMARY ===")
# print(mod_c_minor.summary2())
print("\n\n=== Model C (Minor FE) — selected coefficients ===")
print(coef_table(mod_c_minor, vars_of_interest))



=== Model A — selected coefficients ===
                     coef   se      t    p  ci_low  ci_high
fragmentation_index -1.48 0.04 -33.68 0.00   -1.56    -1.39
human_E1_fraction    0.26 0.02  13.78 0.00    0.22     0.30


=== Model B (Major FE) — selected coefficients ===
                     coef   se      t    p  ci_low  ci_high
fragmentation_index -1.29 0.04 -28.76 0.00   -1.38    -1.20
human_E1_fraction    0.13 0.02   5.09 0.00    0.08     0.17


=== Model C (Minor FE) — selected coefficients ===
                     coef   se      t    p  ci_low  ci_high
fragmentation_index -1.24 0.05 -24.66 0.00   -1.33    -1.14
human_E1_fraction    0.11 0.03   3.84 0.00    0.05     0.17
